In [4]:
import numpy as np
import tensorflow as tf
import pandas as pd
import glob
from tqdm import tqdm
from music21 import *
from tensorflow.python.ops import control_flow_ops
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [5]:
notes = []
for file in glob.glob("*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))

In [6]:
n_vocab=len(set(notes))

In [7]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

In [8]:
model = Sequential()
model.add(LSTM(256,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=10, verbose=1, batch_size=64, callbacks=callbacks_list)

Epoch 1/10


20032/34822 [================>.............] - ETA: 1435s - loss: 4.88 - ETA: 1418s - loss: 4.75 - ETA: 1417s - loss: 4.74 - ETA: 1419s - loss: 4.77 - ETA: 1420s - loss: 4.74 - ETA: 1426s - loss: 4.75 - ETA: 1422s - loss: 4.75 - ETA: 1419s - loss: 4.76 - ETA: 1416s - loss: 4.77 - ETA: 1414s - loss: 4.77 - ETA: 1413s - loss: 4.75 - ETA: 1411s - loss: 4.75 - ETA: 1409s - loss: 4.76 - ETA: 1407s - loss: 4.75 - ETA: 1406s - loss: 4.74 - ETA: 1404s - loss: 4.74 - ETA: 1404s - loss: 4.73 - ETA: 1403s - loss: 4.72 - ETA: 1402s - loss: 4.74 - ETA: 1400s - loss: 4.73 - ETA: 1401s - loss: 4.73 - ETA: 1400s - loss: 4.73 - ETA: 1399s - loss: 4.73 - ETA: 1398s - loss: 4.72 - ETA: 1397s - loss: 4.72 - ETA: 1397s - loss: 4.72 - ETA: 1396s - loss: 4.72 - ETA: 1394s - loss: 4.71 - ETA: 1392s - loss: 4.72 - ETA: 1390s - loss: 4.71 - ETA: 1388s - loss: 4.72 - ETA: 1386s - loss: 4.72 - ETA: 1385s - loss: 4.71 - ETA: 1383s - loss: 4.72 - ETA: 1381s - loss: 4.72 - ETA: 1380s - loss: 4.72 - ETA: 1378s - loss

34822/34822 [==============================] - ETA: 659s - loss: 4.676 - ETA: 656s - loss: 4.676 - ETA: 653s - loss: 4.676 - ETA: 650s - loss: 4.676 - ETA: 648s - loss: 4.675 - ETA: 645s - loss: 4.676 - ETA: 642s - loss: 4.676 - ETA: 639s - loss: 4.676 - ETA: 636s - loss: 4.676 - ETA: 634s - loss: 4.676 - ETA: 631s - loss: 4.676 - ETA: 628s - loss: 4.676 - ETA: 625s - loss: 4.676 - ETA: 622s - loss: 4.676 - ETA: 619s - loss: 4.676 - ETA: 617s - loss: 4.675 - ETA: 614s - loss: 4.675 - ETA: 611s - loss: 4.675 - ETA: 608s - loss: 4.675 - ETA: 605s - loss: 4.675 - ETA: 602s - loss: 4.674 - ETA: 599s - loss: 4.674 - ETA: 597s - loss: 4.674 - ETA: 594s - loss: 4.674 - ETA: 591s - loss: 4.674 - ETA: 588s - loss: 4.674 - ETA: 585s - loss: 4.674 - ETA: 582s - loss: 4.674 - ETA: 579s - loss: 4.674 - ETA: 577s - loss: 4.673 - ETA: 574s - loss: 4.673 - ETA: 571s - loss: 4.673 - ETA: 568s - loss: 4.672 - ETA: 565s - loss: 4.672 - ETA: 562s - loss: 4.672 - ETA: 559s - loss: 4.672 - ETA: 557s - loss:

20096/34822 [================>.............] - ETA: 1596s - loss: 4.60 - ETA: 1652s - loss: 4.59 - ETA: 1626s - loss: 4.52 - ETA: 1610s - loss: 4.56 - ETA: 1594s - loss: 4.53 - ETA: 1587s - loss: 4.50 - ETA: 1576s - loss: 4.49 - ETA: 1569s - loss: 4.48 - ETA: 1563s - loss: 4.48 - ETA: 1558s - loss: 4.50 - ETA: 1554s - loss: 4.48 - ETA: 1550s - loss: 4.48 - ETA: 1548s - loss: 4.48 - ETA: 1543s - loss: 4.48 - ETA: 1539s - loss: 4.48 - ETA: 1537s - loss: 4.48 - ETA: 1533s - loss: 4.49 - ETA: 1532s - loss: 4.48 - ETA: 1529s - loss: 4.49 - ETA: 1526s - loss: 4.49 - ETA: 1524s - loss: 4.50 - ETA: 1520s - loss: 4.50 - ETA: 1517s - loss: 4.50 - ETA: 1514s - loss: 4.49 - ETA: 1511s - loss: 4.49 - ETA: 1509s - loss: 4.49 - ETA: 1505s - loss: 4.49 - ETA: 1502s - loss: 4.49 - ETA: 1499s - loss: 4.48 - ETA: 1496s - loss: 4.48 - ETA: 1493s - loss: 4.48 - ETA: 1490s - loss: 4.48 - ETA: 1487s - loss: 4.48 - ETA: 1483s - loss: 4.48 - ETA: 1480s - loss: 4.49 - ETA: 1478s - loss: 4.49 - ETA: 1475s - loss

34822/34822 [==============================] - ETA: 661s - loss: 4.542 - ETA: 658s - loss: 4.543 - ETA: 655s - loss: 4.542 - ETA: 652s - loss: 4.542 - ETA: 649s - loss: 4.542 - ETA: 646s - loss: 4.542 - ETA: 644s - loss: 4.542 - ETA: 641s - loss: 4.544 - ETA: 638s - loss: 4.545 - ETA: 635s - loss: 4.545 - ETA: 632s - loss: 4.546 - ETA: 629s - loss: 4.546 - ETA: 626s - loss: 4.546 - ETA: 623s - loss: 4.546 - ETA: 620s - loss: 4.547 - ETA: 617s - loss: 4.546 - ETA: 614s - loss: 4.547 - ETA: 612s - loss: 4.546 - ETA: 609s - loss: 4.547 - ETA: 606s - loss: 4.547 - ETA: 603s - loss: 4.547 - ETA: 600s - loss: 4.547 - ETA: 597s - loss: 4.548 - ETA: 594s - loss: 4.548 - ETA: 591s - loss: 4.549 - ETA: 589s - loss: 4.549 - ETA: 586s - loss: 4.549 - ETA: 583s - loss: 4.549 - ETA: 580s - loss: 4.551 - ETA: 577s - loss: 4.552 - ETA: 574s - loss: 4.552 - ETA: 572s - loss: 4.551 - ETA: 569s - loss: 4.551 - ETA: 566s - loss: 4.552 - ETA: 563s - loss: 4.551 - ETA: 560s - loss: 4.551 - ETA: 557s - loss:

20096/34822 [================>.............] - ETA: 1700s - loss: 4.63 - ETA: 1620s - loss: 4.59 - ETA: 1598s - loss: 4.53 - ETA: 1577s - loss: 4.48 - ETA: 1570s - loss: 4.44 - ETA: 1565s - loss: 4.45 - ETA: 1558s - loss: 4.45 - ETA: 1553s - loss: 4.46 - ETA: 1547s - loss: 4.47 - ETA: 1544s - loss: 4.49 - ETA: 1540s - loss: 4.50 - ETA: 1538s - loss: 4.48 - ETA: 1539s - loss: 4.48 - ETA: 1537s - loss: 4.48 - ETA: 1536s - loss: 4.47 - ETA: 1536s - loss: 4.50 - ETA: 1533s - loss: 4.50 - ETA: 1532s - loss: 4.51 - ETA: 1531s - loss: 4.50 - ETA: 1529s - loss: 4.49 - ETA: 1525s - loss: 4.50 - ETA: 1520s - loss: 4.50 - ETA: 1516s - loss: 4.50 - ETA: 1512s - loss: 4.50 - ETA: 1508s - loss: 4.50 - ETA: 1505s - loss: 4.50 - ETA: 1501s - loss: 4.50 - ETA: 1498s - loss: 4.50 - ETA: 1493s - loss: 4.50 - ETA: 1489s - loss: 4.50 - ETA: 1485s - loss: 4.50 - ETA: 1481s - loss: 4.50 - ETA: 1479s - loss: 4.50 - ETA: 1475s - loss: 4.49 - ETA: 1472s - loss: 4.50 - ETA: 1468s - loss: 4.53 - ETA: 1464s - loss

34822/34822 [==============================] - ETA: 657s - loss: 4.494 - ETA: 654s - loss: 4.494 - ETA: 651s - loss: 4.493 - ETA: 648s - loss: 4.493 - ETA: 645s - loss: 4.493 - ETA: 642s - loss: 4.493 - ETA: 639s - loss: 4.492 - ETA: 636s - loss: 4.492 - ETA: 634s - loss: 4.491 - ETA: 631s - loss: 4.491 - ETA: 628s - loss: 4.491 - ETA: 625s - loss: 4.491 - ETA: 622s - loss: 4.490 - ETA: 619s - loss: 4.491 - ETA: 616s - loss: 4.490 - ETA: 613s - loss: 4.489 - ETA: 610s - loss: 4.489 - ETA: 608s - loss: 4.490 - ETA: 605s - loss: 4.489 - ETA: 602s - loss: 4.490 - ETA: 599s - loss: 4.489 - ETA: 596s - loss: 4.489 - ETA: 593s - loss: 4.489 - ETA: 590s - loss: 4.489 - ETA: 588s - loss: 4.489 - ETA: 585s - loss: 4.489 - ETA: 582s - loss: 4.489 - ETA: 579s - loss: 4.489 - ETA: 576s - loss: 4.489 - ETA: 573s - loss: 4.489 - ETA: 570s - loss: 4.489 - ETA: 567s - loss: 4.489 - ETA: 565s - loss: 4.489 - ETA: 562s - loss: 4.489 - ETA: 559s - loss: 4.489 - ETA: 556s - loss: 4.488 - ETA: 553s - loss:

14464/34822 [===========>..................] - ETA: 1558s - loss: 4.43 - ETA: 1554s - loss: 4.35 - ETA: 1545s - loss: 4.39 - ETA: 1543s - loss: 4.37 - ETA: 1542s - loss: 4.39 - ETA: 1539s - loss: 4.37 - ETA: 1538s - loss: 4.38 - ETA: 1535s - loss: 4.40 - ETA: 1533s - loss: 4.42 - ETA: 1532s - loss: 4.41 - ETA: 1529s - loss: 4.40 - ETA: 1526s - loss: 4.40 - ETA: 1522s - loss: 4.40 - ETA: 1519s - loss: 4.40 - ETA: 1516s - loss: 4.40 - ETA: 1513s - loss: 4.42 - ETA: 1510s - loss: 4.43 - ETA: 1508s - loss: 4.43 - ETA: 1507s - loss: 4.43 - ETA: 1503s - loss: 4.42 - ETA: 1499s - loss: 4.42 - ETA: 1498s - loss: 4.44 - ETA: 1494s - loss: 4.43 - ETA: 1491s - loss: 4.44 - ETA: 1488s - loss: 4.44 - ETA: 1484s - loss: 4.43 - ETA: 1483s - loss: 4.42 - ETA: 1479s - loss: 4.43 - ETA: 1477s - loss: 4.43 - ETA: 1474s - loss: 4.43 - ETA: 1471s - loss: 4.43 - ETA: 1468s - loss: 4.43 - ETA: 1465s - loss: 4.43 - ETA: 1462s - loss: 4.43 - ETA: 1459s - loss: 4.42 - ETA: 1456s - loss: 4.43 - ETA: 1453s - loss

In [ ]:
model = Sequential()
model.add(LSTM(512,input_shape=(network_input.shape[1], network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')


In [ ]:
# Load the weights to each node
#model.load_weights('weights-improvement-09-3.2697-bigger.hdf5')

In [ ]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start]
prediction_output = []
pattern=list(pattern)
# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]

In [ ]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output.mid')